In [6]:
%load_ext autoreload
%autoreload
import torch
import torch.nn as nn
from collections import OrderedDict
from models.SceneNetStereoJointIter import SceneNetStereoJointIter
from models.SceneNetMonoJointIter import SceneNetMonoJointIter
from datasets.kitti_2015_train import KITTI_2015_MonoSceneFlow_Full
from datasets.kitti_raw_monosf import KITTI_Odom_Test

from losses_eval import Eval_SceneFlow_KITTI_Test, Eval_SceneFlow_KITTI_Train
from losses import Loss_SceneFlow_SelfSup_JointIter

from augmentations import Augmentation_Resize_Only
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from tqdm import tqdm
from pprint import pprint
import numpy as np

class Args:
    model_name='scenenet_joint_stereo_iter'
    cuda = True
    use_bn = True
    momentum = 0.9
    beta = 0.999
    weight_decay=0.0
    use_mask = True
    sf_lr_w = 0.0
    pose_lr_w = 0.0
    mask_lr_w = 1.0
    disp_lr_w = 1.0
    disp_pts_w = 0.0
    sf_pts_w = 0.2
    sf_sm_w = 200
    fb_w = 0.2
    pose_sm_w = 200
    pose_pts_w = 0.2
    disp_sm_w = 0.2
    disp_smooth_w = 0.1
    mask_reg_w = 0.2
    encoder_name="pwc"
    static_cons_w = 1.0
    mask_cons_w = 0.2
    mask_sm_w = 0.1
    flow_diff_thresh=1e-3
    evaluation=True
    finetuning=False

args = Args()

model = SceneNetMonoJointIter(args).cuda()
state_dict = torch.load('pretrained/fb/6.ckpt')['model']
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:]
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model.eval()

dataset_09 = KITTI_Odom_Test(args, "/external/datasets/kitti_data_jpg/", "09")
dataloader_09 = DataLoader(dataset_09, shuffle=False, batch_size=1, pin_memory=True)
dataset_10 = KITTI_Odom_Test(args, "/external/datasets/kitti_data_jpg/", "10")
dataloader_10 = DataLoader(dataset_10, shuffle=False, batch_size=1, pin_memory=True)

aug = Augmentation_Resize_Only(args).cuda()

In [16]:
# from https://github.com/tinghuiz/SfMLearner
def compute_ate(gtruth_xyz, pred_xyz):
    alignment_error = pred_xyz - gtruth_xyz
    rmse = np.sqrt(sum(alignment_error ** 2)) / gtruth_xyz.shape[0]
    
    return rmse

ates = []
for i, data in enumerate(tqdm(dataloader_09)):
    with torch.no_grad():
        # Get input and target tensor keys
        input_keys = list(filter(lambda x: "input" in x, data.keys()))
        target_keys = list(filter(lambda x: "target" in x, data.keys()))
        tensor_keys = input_keys + target_keys
        
        # Possibly transfer to Cuda
        for k, v in data.items():
            if k in tensor_keys:
                data[k] = v.cuda(non_blocking=True)
                
        aug_data = aug(data)
        out = model(aug_data)
        pose = out['pose_f'][0][:, :3, 3].double().cpu()
        gt_t = data['target_pose'][:, :3, 3].cpu()
        
        print(f"pose_f: {pose}")
        print(f"gt    : {gt_t}")
        ate = compute_ate(gt_t, pose)
        ates.append(np.array(ate))
        
        if i == 15:
            break
        
ates = np.array(ates)
print("\n   Trajectory error: {:0.3f}, std: {:0.3f}\n".format(np.mean(ates), np.std(ates)))

# save_path = os.path.join(opt.load_weights_folder, "poses.npy")
# np.save(save_path, pred_poses)
# print("-> Predictions saved to", save_path)

  0%|          | 2/1590 [00:00<05:22,  4.93it/s]

pose_f: tensor([[-0.0155,  0.0058, -0.1012]], dtype=torch.float64)
gt    : tensor([[ 0.0214, -0.0085,  0.2881]], dtype=torch.float64)
pose_f: tensor([[-0.0109,  0.0048, -0.0911]], dtype=torch.float64)
gt    : tensor([[ 0.0140, -0.0071,  0.2928]], dtype=torch.float64)


  0%|          | 4/1590 [00:00<05:17,  5.00it/s]

pose_f: tensor([[-0.0042,  0.0048, -0.0865]], dtype=torch.float64)
gt    : tensor([[ 0.0098, -0.0054,  0.2987]], dtype=torch.float64)
pose_f: tensor([[-9.5583e-05,  5.4669e-03, -1.0602e-01]], dtype=torch.float64)
gt    : tensor([[ 0.0046, -0.0048,  0.3061]], dtype=torch.float64)


  0%|          | 6/1590 [00:01<05:14,  5.04it/s]

...

pose_f: tensor([[ 0.0144,  0.0102, -0.1751]], dtype=torch.float64)
gt    : tensor([[-0.0240, -0.0073,  0.4019]], dtype=torch.float64)
pose_f: tensor([[ 0.0136,  0.0102, -0.1732]], dtype=torch.float64)
gt    : tensor([[-0.0231, -0.0074,  0.4168]], dtype=torch.float64)


  1%|          | 15/1590 [00:02<05:10,  5.07it/s]

pose_f: tensor([[ 0.0088,  0.0088, -0.1734]], dtype=torch.float64)
gt    : tensor([[-0.0207, -0.0077,  0.4283]], dtype=torch.float64)
pose_f: tensor([[ 0.0053,  0.0089, -0.1826]], dtype=torch.float64)
gt    : tensor([[-0.0137, -0.0075,  0.4425]], dtype=torch.float64)


  1%|          | 15/1590 [00:03<05:30,  4.76it/s]


   Trajectory error: 0.179, std: 0.229

